In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os, sys

from sklearn import tree
from sklearn import metrics  

%matplotlib inline
plt.style.use('seaborn-white')

In [7]:
from distutils.version import LooseVersion
from sklearn import __version__ as sklearn_version

if LooseVersion(sklearn_version) < '0.18':
    from sklearn.cross_validation import train_test_split, cross_val_score
else:
    from sklearn.model_selection import train_test_split, cross_val_score

In [8]:
os.getcwd()
os.chdir('C://Users/Michael Hong/Desktop/VIVA/datasheet')

In [9]:
ran = pd.read_csv('전처리완료2.csv',engine ='python')
ran.info()
ran.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18398 entries, 0 to 18397
Data columns (total 10 columns):
활성도_지표_구간_코드          18398 non-null int64
안정성_지표_구간_코드          18398 non-null int64
서비스_업종_코드_명_커피음료      18398 non-null int64
성장성_지표_구간_코드          18398 non-null int64
서비스_업종_코드_명_분식집       18398 non-null int64
서비스_업종_코드_명_패스트푸드점    18398 non-null int64
서비스_업종_코드_명_치킨집       18398 non-null int64
cluster               18398 non-null int64
중년층_직장인구_수            18398 non-null int64
과밀_지수_구간_코드           18398 non-null float64
dtypes: float64(1), int64(9)
memory usage: 1.4 MB


,활성도_지표_구간_코드,안정성_지표_구간_코드,서비스_업종_코드_명_커피음료,성장성_지표_구간_코드,서비스_업종_코드_명_분식집,서비스_업종_코드_명_패스트푸드점,서비스_업종_코드_명_치킨집,cluster,중년층_직장인구_수,과밀_지수_구간_코드
0,2,1,0,2,0,0,0,3,84,2.0
1,3,3,0,3,0,0,0,0,84,1.0
2,2,0,1,2,0,0,0,0,84,0.0
3,3,3,0,3,0,0,0,0,84,1.0
4,2,0,0,3,0,0,1,0,84,0.0


In [10]:
X = ran.drop('과밀_지수_구간_코드', axis=1)
Y = ran['과밀_지수_구간_코드']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

model = RandomForestClassifier(min_samples_split=10, criterion="entropy", n_estimators=50,)


#train model
model.fit(X_train, Y_train)

Y_pred = model.predict(X_test)

#print ("FINISHED classifying. accuracy score : ")
print('Accuracy : %.2f' %accuracy_score(Y_test,Y_pred))

Accuracy : 0.69


In [15]:
ran.columns

Index(['활성도_지표_구간_코드', '안정성_지표_구간_코드', '서비스_업종_코드_명_커피음료', '성장성_지표_구간_코드',
       '서비스_업종_코드_명_분식집', '서비스_업종_코드_명_패스트푸드점', '서비스_업종_코드_명_치킨집', 'cluster',
       '중년층_직장인구_수', '과밀_지수_구간_코드'],
      dtype='object')

In [18]:
model.get_params()

{'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 10,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 50,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [24]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

In [25]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}

rf = RandomForestClassifier()

In [26]:
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [27]:
grid_search.fit(X_train, Y_train)
grid_search.best_params_
{'bootstrap': True,
 'max_depth': 80,
 'max_features': 3,
 'min_samples_leaf': 5,
 'min_samples_split': 12,
 'n_estimators': 100}
best_grid = grid_search.best_estimator_
grid_accuracy = evaluate(best_grid, X_test, Y_test)

Fitting 3 folds for each of 288 candidates, totalling 864 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   49.1s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  9.7min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed: 17.9min
[Parallel(n_jobs=-1)]: Done 864 out of 864 | elapsed: 24.3min finished


Model Performance
Average Error: 0.3377 degrees.
Accuracy = -inf%.
